In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.3 MB/s eta 0:00:00


**Now** **Import Models**

In [2]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


 Licence Palet Detection Model 1

In [ ]:
Model1 = YOLO('/content/drive/MyDrive/car number plates/no and text/dataset of cropped img/LP.pt')

OCR Model 2

In [ ]:
Model2 = YOLO('/content/drive/MyDrive/car number plates/no and text/dataset of cropped img/best (1).pt')

**For Image**

In [ ]:
imgpath = "/content/drive/MyDrive/car number plates/FINAL TESTING/WhatsApp Image 2025-10-30 at 17.43.06_3bb4a9e0.jpg"

Prediction if Model 1

In [ ]:
LP = Model1.predict(imgpath, conf=0.25)

Cropping, OCR and Listing

In [ ]:
import cv2

In [ ]:
image = cv2.imread(imgpath)
listlp = []
for i, box in enumerate(LP[0].boxes.xyxy.cpu().numpy()):
  x1,y1,x2,y2 = map(int, box)
  cropped = image[y1:y2, x1:x2]
  ocr = Model2.predict(cropped, conf=0.25)
  listlp.append(ocr[0].boxes)

Sorting Number

In [ ]:
for boxes in listlp:
    class_ids = boxes.cls.cpu().numpy().astype(int)
    xyxy = boxes.xyxy.cpu().numpy()
    class_names = Model2.names

    # Compute centers
    box_info = []
    for xy, cid in zip(xyxy, class_ids):
        x1, y1, x2, y2 = xy
        cx = (x1 + x2) / 2  # center X
        cy = (y1 + y2) / 2  # center Y
        box_info.append((cx, cy, cid))

    # Sort top-to-bottom, then left-to-right
    line_tolerance = 20
    box_info.sort(key=lambda b: (round(b[1] / line_tolerance), b[0]))

    # 🔄 Reverse if OCR boxes are detected right-to-left
    if len(box_info) >= 2 and box_info[0][0] > box_info[-1][0]:
        box_info.reverse()

    # Build text using sorted boxes
    ordered_number = ''.join(class_names[cid] for _, _, cid in box_info)

In [ ]:
print(f"Car {i} Licence Number is: {ordered_number}")

In [ ]:
import cv2
import matplotlib.pyplot as plt
result = ocr[0]
# result.plot()  # This will plot the image with bounding boxes on it
# result = results[0]

# Get the image and bounding boxes
img = result.orig_img  # This contains the image array
boxes = result.boxes  # Bounding box coordinates (Box class object)
labels = result.names  # Class labels

# Convert BGR to RGB for proper color display in matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Plotting the image with bounding boxes
plt.imshow(img_rgb)
plt.axis('off')  # Hide axes

# Iterate over the boxes and draw rectangles
for i, box in enumerate(boxes):
    # Extract box coordinates (x1, y1, x2, y2) and confidence score
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Ensure this is a numpy array
    confidence = box.conf[0].cpu().numpy()  # Confidence score
    label_idx = int(box.cls[0].cpu().numpy())  # Class index

    # Draw the bounding box
    cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Display the class label and confidence score
    label = labels[label_idx]  # Get the class label name from the index
    cv2.putText(img_rgb, f'{label} {confidence:.2f}', (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Show the image with bounding boxes
plt.imshow(img_rgb)
plt.show()

**For Video**

In [ ]:
input_video = '/content/drive/MyDrive/car number plates/FINAL TESTING/WhatsApp Video 2025-10-23 at 22.08.22_53776f31.mp4'
output_video = '/content/drive/MyDrive/car number plates/FINAL TESTING/output.mp4'

# Open video
cap = cv2.VideoCapture(input_video)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Create output video writer
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
listlp = []
# Process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO on the frame
    results = Model1(frame, conf=0.25)
    #Cropped and OCR
    #image = cv2.imread(frame)

    for i, box in enumerate(results[0].boxes.xyxy.cpu().numpy()):
      x1,y1,x2,y2 = map(int, box)
      cropped = frame[y1:y2, x1:x2]
      ocr = Model2.predict(cropped, conf=0.25)
      listlp.append(ocr[0].boxes)


    annotated_frame = results[0].plot()
    out.write(annotated_frame)

# Release resources
cap.release()
out.release()

print("✅ Done! Processed video saved at:", output_video)


Sorting into List

In [ ]:
# Sorting
list_for_numbers = []
for boxes in listlp:
    class_ids = boxes.cls.cpu().numpy().astype(int)
    xyxy = boxes.xyxy.cpu().numpy()
    class_names = Model2.names

    # Compute centers
    box_info = []
    for xy, cid in zip(xyxy, class_ids):
        x1, y1, x2, y2 = xy
        cx = (x1 + x2) / 2  # center X
        cy = (y1 + y2) / 2  # center Y
        box_info.append((cx, cy, cid))

    # Sort top-to-bottom, then left-to-right
    line_tolerance = 20
    box_info.sort(key=lambda b: (round(b[1] / line_tolerance), b[0]))

    # 🔄 Reverse if OCR boxes are detected right-to-left
    if len(box_info) >= 2 and box_info[0][0] > box_info[-1][0]:
        box_info.reverse()

    # Build text using sorted boxes
    ordered_number = ''.join(class_names[cid] for _, _, cid in box_info)
    list_for_numbers.append(ordered_number)


In [ ]:
print(list_for_numbers)

Checking for a Specific Number

In [ ]:
for i in list_for_numbers:
  if i == '':
    print("Founded the Car Number plate: "+ str(i))